In [10]:
import nltk
from nltk.tag import CRFTagger
import os
import numpy as np
import io

In [2]:
# Prepare dataset for NLTK HMM Tagger
# Divide in train and test files [80:20] 

# Directory having content
doc_dir = 'Data/content'

train_file_list = []
test_file_list = []

for f in os.listdir(doc_dir):
    #Random Sampling
    if np.random.uniform(0,1)< 0.8:
        train_file_list.append(f)
    else:
        test_file_list.append(f)

print len(train_file_list)
print len(test_file_list)

1035
272


In [11]:
# Build Tagged dataset to be fed in nltk hmm tagger
# It is a list of tagged sentences. 
# training_data = [ [(word, tag), (word, tag).....]
# [(word, tag), (word, tag).....]
#]

# Directory having tags
tag_dir = 'Data/new_tags'

training_data = []

for f in train_file_list:
    training_sentences =[]
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with io.open(word_file_path, "rt", encoding="utf-8") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with io.open(tag_file_path, "rt", encoding="utf-8") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            pairs_in_line = []
            length = min(len(word_in_file), len(tag_in_file))
            #Create the word_tag pair
            for j in xrange(length):
                pairs_in_line.append((word_in_file[j], tag_in_file[j]));
            training_sentences.append(pairs_in_line)
    if len(training_sentences) > 0:
        training_data.extend(training_sentences)

#print training_data[0]
            

In [12]:
ct = CRFTagger()
ct.train(training_data,'model.crf.tagger')

In [36]:
for f in test_file_list:
    word_file_path = os.path.join(doc_dir, f)
    with io.open(word_file_path, "rt", encoding="utf-8") as word_file:
        for line in word_file:
            predicted_tags = ct.tag_sents([line.split()])
            print predicted_tags[0]
    break

[(u'Atiq', 'PER_Others'), (u'Khan', 'PER_Others'), (u'.', 'O'), (u'.', 'O'), (u'LUCKNOW', 'LOC_Others'), (u':', 'O'), (u'The', 'O'), (u'Anti-Terrorism', 'ORG_Others'), (u'Squad', 'ORG_Others'), (u'-LRB-', 'O'), (u'ATS', 'O'), (u'-RRB-', 'O'), (u',', 'O'), (u'Mumbai', 'LOC_Others'), (u',', 'O'), (u'on', 'O'), (u'Thursday', 'O'), (u'secured', 'O'), (u'the', 'O'), (u'transit', 'O'), (u'remand', 'O'), (u'for', 'O'), (u'Sudhakar', 'PER_Others'), (u'Dwivedi', 'PER_Others'), (u',', 'O'), (u'alias', 'O'), (u'Amritanand', 'O'), (u',', 'O'), (u'a', 'O'), (u'suspect', 'O'), (u'in', 'O'), (u'the', 'O'), (u'Malegaon', 'LOC_Event'), (u'bomb', 'O'), (u'blast', 'O'), (u'case', 'O'), (u'of', 'O'), (u'September', 'O'), (u',', 'O'), (u'2008', 'O'), (u'.', 'O'), (u'The', 'O'), (u'remand', 'O'), (u'till', 'O'), (u'November', 'O'), (u'16', 'O'), (u'was', 'O'), (u'granted', 'O'), (u'by', 'O'), (u'Mukesh', 'PER_Others'), (u'Kumar', 'PER_Others'), (u',', 'O'), (u'Civil', 'O'), (u'Judge', 'O'), (u',', 'O'), (u'

In [50]:
# Test
actual_tag_count = dict()
matched_tag_count = dict()
matched_tag_count[u'PER_Others'] = 0
actual_tag_count[u'PER_Others'] = 0
matched_tag_count[u'PER_Victim'] = 0
actual_tag_count[u'PER_Victim'] = 0
matched_tag_count[u'PER_Accused'] = 0
actual_tag_count[u'PER_Accused'] = 0
matched_tag_count[u'ORG_Victim'] = 0
actual_tag_count[u'ORG_Victim'] = 0
matched_tag_count[u'ORG_Accused'] = 0
actual_tag_count[u'ORG_Accused'] = 0
matched_tag_count[u'ORG_Others'] = 0
actual_tag_count[u'ORG_Others'] = 0
matched_tag_count[u'LOC_Accused'] = 0
actual_tag_count[u'LOC_Accused'] = 0
matched_tag_count[u'LOC_Others'] = 0
actual_tag_count[u'LOC_Others'] = 0
matched_tag_count[u'LOC_Event'] = 0
actual_tag_count[u'LOC_Event'] = 0
matched_tag_count[u'LOC_Victim'] = 0
actual_tag_count[u'LOC_Victim'] = 0

for f in test_file_list:
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    predicted_tags = []
    with io.open(word_file_path, "rt", encoding="utf-8") as word_file:
        for line in word_file:
            predicted_tags = ct.tag_sents([line.split()])
    if len(predicted_tags[0]) > 0:
        actual_tags = []
        with io.open(tag_file_path, "rt", encoding="utf-8") as tag_file:
            for line in tag_file:
                #print(line + '\n')
                actual_tags = line.split()
                #print actual_tags
        result_len = min(len(predicted_tags[0]), len(actual_tags))
        
        for i in xrange(result_len):
            #print predicted_tags[i][1], actual_tags[i]
            if actual_tags[i] in actual_tag_count.keys():
                actual_tag_count[actual_tags[i]] = actual_tag_count[actual_tags[i]] + 1
                
            if actual_tags[i] == predicted_tags[0][i][1]:
                if actual_tags[i] in matched_tag_count.keys():
                    matched_tag_count[actual_tags[i]] = matched_tag_count[actual_tags[i]] + 1

print actual_tag_count

{u'PER_Others': 1970, u'LOC_Event': 1467, u'LOC_Others': 1822, u'ORG_Accused': 605, u'PER_Victim': 256, u'ORG_Victim': 69, u'ORG_Others': 2510, u'PER_Accused': 947, u'LOC_Accused': 173, u'LOC_Victim': 34}


In [51]:
print('======= Accuracy Class Wise =====================\n')

print('Class    Matched Total %')
print('--------------------------')
print ('PER_Others: '+str(matched_tag_count['PER_Others'])+ ' ' + str(actual_tag_count['PER_Others'])+ ' ' + str(matched_tag_count['PER_Others']*100/actual_tag_count['PER_Others'])+'%')
print ('PER_Victim: '+str(matched_tag_count['PER_Victim'])+ ' ' + str(actual_tag_count['PER_Victim'])+ ' '+str(matched_tag_count['PER_Victim']*100/actual_tag_count['PER_Victim'])+'%')
print ('PER_Accused: '+str(matched_tag_count['PER_Accused'])+ ' ' + str(actual_tag_count['PER_Accused'])+ ' '+str(matched_tag_count['PER_Accused']*100/actual_tag_count['PER_Accused'])+'%')
print ('ORG_Victim: '+str(matched_tag_count['ORG_Victim']) + ' '+ str(actual_tag_count['ORG_Victim'])+ ' '+str(matched_tag_count['ORG_Victim']*100/actual_tag_count['ORG_Victim'])+'%')
print ('ORG_Accused: '+str(matched_tag_count['ORG_Accused']) + ' '+ str(actual_tag_count['ORG_Accused'])+ ' '+str(matched_tag_count['ORG_Accused']*100/actual_tag_count['ORG_Accused'])+'%')
print ('ORG_Others: '+str(matched_tag_count['ORG_Others']) + ' '+ str(actual_tag_count['ORG_Others'])+ ' '+str(matched_tag_count['ORG_Others']*100/actual_tag_count['ORG_Others'])+'%')
print ('LOC_Accused: '+str(matched_tag_count['LOC_Accused'])+ ' ' + str(actual_tag_count['LOC_Accused'])+ ' '+str(matched_tag_count['LOC_Accused']*100/actual_tag_count['LOC_Accused'])+'%')
print ('LOC_Others: '+str(matched_tag_count['LOC_Others'])+ ' ' + str(actual_tag_count['LOC_Others'])+ ' '+str(matched_tag_count['LOC_Others']*100/actual_tag_count['LOC_Others'])+'%')
print ('LOC_Event: '+str(matched_tag_count['LOC_Event']) + ' '+ str(actual_tag_count['LOC_Event'])+ ' '+str(matched_tag_count['LOC_Event']*100/actual_tag_count['LOC_Event'])+'%')
print ('LOC_Victim: '+str(matched_tag_count['LOC_Victim']) + ' '+ str(actual_tag_count['LOC_Victim'])+ ' '+str(matched_tag_count['LOC_Victim']*100/actual_tag_count['LOC_Victim'])+'%')

======= Accuracy Class Wise =====================

Class    Matched Total %
--------------------------
PER_Others: 1407 1970 71%
PER_Victim: 8 256 3%
PER_Accused: 293 947 30%
ORG_Victim: 0 69 0%
ORG_Accused: 461 605 76%
ORG_Others: 1257 2510 50%
LOC_Accused: 0 173 0%
LOC_Others: 971 1822 53%
LOC_Event: 622 1467 42%
LOC_Victim: 0 34 0%
